In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import regex as re
from PIL import Image
import pytesseract as ocr
from ISR.models import RRDN
from google.colab.patches import cv2_imshow
from pdf2image import convert_from_path
import img2pdf

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip3 install ISR
!pip install pdf2image
!pip install img2pdf
!sudo apt-get install poppler-utils
!pip install regex
!pip install ISR --no-deps

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 7 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (11.2 MB/s)
debconf: unable to initial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 312 kB 9.7 MB/s 
     |████████████████████████████████| 5.1 MB 54.9 MB/s 
ERROR: Cannot install isr==1.9.0, isr==1.9.1, isr==1.9.2, isr==1.9.3, isr==2.0.0, isr==2.0.1, isr==2.0.2, isr==2.0.5, isr==2.1, isr==2.1.1 and isr==2.2.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    isr 2.2.0 depends on tensorflow==2.0.0
    isr 2.1.1 depends on tensorflow==2.0.0
    isr 2.1 depends on tensorflow==1.13.1
    isr 2.0.5 depends on tensorflow==1.13.1
    isr 2.0.2 depends on tensorflow==1.13.1
    isr 2.0.1 depends on tensorflow==1.13.1
    isr 2.0.0 depends on tensorflow==1.13.1
    isr 1.9.3 depends on tensorflow==1.13.1
    isr 1.9.2 depends on tensorflow==1.13.1
    isr 1.9.1 depends on tensorflow==1.13.1
    isr 1.9.0 depends on tensorflow==1.13.1

To fix this you could try to:
1. loosen the range of package 

In [ ]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 6.2 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [ ]:
SR_Model = RRDN(weights='gans')

17462488/17462488 [==============================] - 1s 0us/step


In [ ]:
multiplication_table = (
    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    (1, 2, 3, 4, 0, 6, 7, 8, 9, 5),
    (2, 3, 4, 0, 1, 7, 8, 9, 5, 6),
    (3, 4, 0, 1, 2, 8, 9, 5, 6, 7),
    (4, 0, 1, 2, 3, 9, 5, 6, 7, 8),
    (5, 9, 8, 7, 6, 0, 4, 3, 2, 1),
    (6, 5, 9, 8, 7, 1, 0, 4, 3, 2),
    (7, 6, 5, 9, 8, 2, 1, 0, 4, 3),
    (8, 7, 6, 5, 9, 3, 2, 1, 0, 4),
    (9, 8, 7, 6, 5, 4, 3, 2, 1, 0))

permutation_table = (
    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    (1, 5, 7, 6, 2, 8, 3, 0, 9, 4),
    (5, 8, 0, 3, 7, 9, 6, 1, 4, 2),
    (8, 9, 1, 6, 0, 4, 3, 5, 2, 7),
    (9, 4, 5, 3, 1, 2, 6, 8, 7, 0),
    (4, 2, 8, 6, 5, 7, 3, 9, 0, 1),
    (2, 7, 9, 3, 8, 0, 6, 4, 1, 5),
    (7, 0, 4, 6, 9, 1, 3, 2, 5, 8))

def compute_checksum(number):
    
    """Calculate the Verhoeff checksum over the provided number. The checksum
    is returned as an int. Valid numbers should have a checksum of 0."""
    
    # transform number list
    number = tuple(int(n) for n in reversed(str(number)))
    
    # calculate checksum
    checksum = 0
    
    for i, n in enumerate(number):
        checksum = multiplication_table[checksum][permutation_table[i % 8][n]]
    
    return checksum

In [ ]:
def Regex_Search(bounding_boxes):

  possible_UIDs = []
  Result = ""

  for character in range(len(bounding_boxes)):
    if len(bounding_boxes[character])!=0:
      Result += bounding_boxes[character][0]
    else:
      Result += '?'

  matches = [match.span() for match in re.finditer(r'\d{12}',Result,overlapped=True)]

  for match in matches :

    UID = int(Result[match[0]:match[1]])
    
    if compute_checksum(UID)==0 and UID%10000!=1947:
       possible_UIDs.append([UID,match[0]])

  possible_UIDs = np.array(possible_UIDs)
  return possible_UIDs

In [ ]:
def Mask_UIDs (image_path,possible_UIDs,bounding_boxes,rtype,SR=False,SR_Ratio=[1,1]):

  img = cv2.imread(image_path)

  if rtype==2:
    img = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)
  elif rtype==3:
    img = cv2.rotate(img,cv2.ROTATE_180)
  elif rtype==4:
    img = cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)

  height = img.shape[0]

  if SR==True:
    height*=SR_Ratio[1]

  ########################### MASK EVERY DIGIT INDIVIDUALLY ############################

  # for UID in possible_UIDs:

  #   for i in range(8):

  #     digit = bounding_boxes[UID[1]+i].split()

  #     if SR==False:
  #       top_left_corner = (int(digit[1]),height-int(digit[4]))
  #       bottom_right_corner = (int(digit[3]),height-int(digit[2]))

  #     else:
  #       top_left_corner = (int(int(digit[1])/SR_Ratio[0]),int((height-int(digit[4]))/SR_Ratio[1]))
  #       bottom_right_corner = (int(int(digit[3])/SR_Ratio[0]),int((height-int(digit[2]))/SR_Ratio[1]))

  #     img = cv2.rectangle(img,top_left_corner,bottom_right_corner,(0,0,0),-1)

  ######################################################################################

  for UID in possible_UIDs:

    digit1 = bounding_boxes[UID[1]].split()
    digit8 = bounding_boxes[UID[1] + 7].split()

    h1 = min(height-int(digit1[4]),height-int(digit8[4]))
    h2 = max(height-int(digit1[2]),height-int(digit8[2]))

    if SR==False:
      top_left_corner = (int(digit1[1]),h1)
      bottom_right_corner = (int(digit8[3]),h2)

    else:
      top_left_corner = (int(int(digit1[1])/SR_Ratio[0]),int((h1)/SR_Ratio[1]))
      bottom_right_corner = (int(int(digit8[3])/SR_Ratio[0]),int((h2)/SR_Ratio[1]))

    img = cv2.rectangle(img,top_left_corner,bottom_right_corner,(0,0,0),-1)

  if rtype==2:
    img = cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)
  elif rtype==3:
    img = cv2.rotate(img,cv2.ROTATE_180)
  elif rtype==4:
    img = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)

  file_name = image_path.split('/')[-1].split('.')[0]+"_masked"+"."+image_path.split('.')[-1]
  cv2.imwrite(file_name,img)
  return file_name

In [ ]:
def Extract_and_Mask_UIDs (image_path,SR=False,sr_image_path=None,SR_Ratio=[1,1]):

  if SR==False:
    img = cv2.imread(image_path)
  else:
    img = cv2.imread(sr_image_path)

  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  rotations = [[gray,1],
               [cv2.rotate(gray,cv2.ROTATE_90_COUNTERCLOCKWISE),2],
               [cv2.rotate(gray,cv2.ROTATE_180),3],
               [cv2.rotate(gray,cv2.ROTATE_90_CLOCKWISE),4],
               [cv2.GaussianBlur(gray,(5,5),0),1],
               [cv2.GaussianBlur(cv2.rotate(gray,cv2.ROTATE_90_COUNTERCLOCKWISE),(5,5),0),2],
               [cv2.GaussianBlur(cv2.rotate(gray,cv2.ROTATE_180),(5,5),0),3],
               [cv2.GaussianBlur(cv2.rotate(gray,cv2.ROTATE_90_CLOCKWISE),(5,5),0),4]]

  settings = ('-l eng --oem 3 --psm 11')

  for rotation in rotations :
    
    cv2.imwrite('rotated_grayscale.png',rotation[0])
    bounding_boxes = ocr.image_to_boxes(Image.open('rotated_grayscale.png'),config=settings).split(" 0\n")

    possible_UIDs = Regex_Search(bounding_boxes)

    if len(possible_UIDs)==0:
      continue
    else:

      if SR==False:
        masked_img = Mask_UIDs (image_path,possible_UIDs,bounding_boxes,rotation[1])
      else:
        masked_img = Mask_UIDs (image_path,possible_UIDs,bounding_boxes,rotation[1],True,SR_Ratio)

      return (masked_img,possible_UIDs)

  return (None,None)

In [ ]:
input_path = "drive/My Drive/data/14.pdf"   # Path to the Input Image/PDF

if input_path.split('.')[-1]=="pdf":
    
    pages = convert_from_path(input_path, 300)
    pages[0].save('pdf2img.jpg', 'JPEG')

if input_path.split('.')[-1]=="pdf":
  masked_img,possible_UIDs = Extract_and_Mask_UIDs('pdf2img.jpg')
else:
  masked_img,possible_UIDs = Extract_and_Mask_UIDs(input_path)

if masked_img!=None and input_path.split('.')[-1]=="pdf":
    
  image = Image.open(masked_img) 
  pdf_bytes = img2pdf.convert(image.filename) 
  file = open(input_path.split('/')[-1].split('.')[0]+"_masked"+".pdf", "wb")
  masked_img = input_path.split('/')[-1].split('.')[0]+"_masked"+".pdf" 
  file.write(pdf_bytes) 
  image.close() 
  file.close() 

if masked_img==None:
  print("Can't find any UID!")
else:
  print("Found UIDs : "+str(possible_UIDs[:,0]))

Found UIDs : [255002305082]
